In [8]:
import Pkg; 
Pkg.add("LinearMaps")
Pkg.add("IterativeSolvers")

   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
   Installed IterativeSolvers ─ v0.9.2
    Updating `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  [42fd0dbc] + IterativeSolvers v0.9.2
    Updating `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
  [42fd0dbc] + IterativeSolvers v0.9.2
Precompiling project...
  ✓ IterativeSolvers
  1 dependency successfully precompiled in 2 seconds. 93 already precompiled.


In [69]:
using LinearMaps
using IterativeSolvers
using FFTW
using Base.Threads
using PaddedViews
using BenchmarkTools
using ChainRulesCore

function g2g̃(g)
    g̃ = complex.(zeros(size(g)),zeros(size(g)))
    n = size(g)[3]
    Threads.@threads for i in 1:n
        g̃[:,:,i] .= fft(g[:,:,i])
    end
    g̃
end

function g2g̃ᵀ(g)
    g̃ᵀ = complex.(zeros(size(g)),zeros(size(g)))
    n = size(g)[3]
    Threads.@threads for i in 1:n
        g̃ᵀ[:,:,i] .= fft(reverse(g[:,:,i]))
    end
    g̃ᵀ
end

⊛(g̃,u) = real.(ifftshift( ifft( g̃ .* fft(u) ) ))
✪(v,u) = real.(ifftshift( ifft( fft(v) .* conj.(fft(u)))))
syz(u,g̃) = collect(sym_paddedviews(0,u,g̃)[1])
v∂gu(g̃,u,v) = ✪(syz(v,g̃),syz(u,g̃))

function conv(u,K̃,fwd::Bool)
    nᵢ, nₖ = size(u)[1], size(K̃)[1]
    nₒ = nₖ-nᵢ

    i₁ = fwd==true ? 1+(nᵢ÷2) : nᵢ÷2
    i₂ = i₁+nₒ-1
    ⊛(K̃,syz(u,K̃))[i₁:i₂,i₁:i₂]
end

function G(u,g̃)
    nᵢ, nₖ = size(u)[1], size(g̃)[1]
    nₒ = nₖ-nᵢ
    Gu = zeros(Float64,nₒ,nₒ)
    n = size(g̃)[3]
    Threads.@threads for i in 1:n
        Gu[:,:] .+= conv(u[:,:,i],g̃[:,:,i],true)
    end

    Gu
end

function GᵀG(u,g̃,g̃ᵀ)
    
    Gu = G(u,g̃)
    n = size(g̃)[3]
    GᵀGu = zeros(size(u))
    for i in 1:n
        GᵀGu[:,:,i] .= conv(Gu,g̃ᵀ[:,:,i],false)
    end

    GᵀGu
end

function y∂GᵀGx(g̃,x,y)

    Gx = G(x,g̃)
    Gy = G(y,g̃)

    n = size(g̃)[3]
    ∂g = zeros(size(g̃))
    Threads.@threads for i in 1:n
        ∂g[:,:,i] .= v∂gu(g̃[:,:,i],x[:,:,i],Gy) .+ v∂gu(g̃[:,:,i],y[:,:,i],Gx)
    end
    
    ∂g
end

GᵀGαI(x,g̃,g̃ᵀ,α) = GᵀG(x,g̃,g̃ᵀ) .+ α.*x

function regress!(x,g̃,g̃ᵀ,α,y)

    dims = size(x)

    x = vec(x)
    y = vec(y)
    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(x)[1],issymmetric=true,isposdef=true)
    cg!(x,Â,y)
    x = reshape(x,dims)
    y = reshape(y,dims)

end

function G2û(g,u,α)

    g̃ = g2g̃(g)
    g̃ᵀ = g2g̃ᵀ(g)
    
    û = zeros(size(u))
    y = GᵀG(u,g̃,g̃ᵀ)
    regress!(û,g̃,g̃ᵀ,α,y)

    û, g̃, g̃ᵀ
end

function û2G(∇ℓ,g̃,g̃ᵀ,u,α,û)

    Λ = zeros(size(u))
    regress!(Λ,g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û), g,u,α)
    û,g̃,g̃ᵀ = G2û(g,u,α)
    function G2û_pullback(∇ℓ)

        ∂û = @thunk(û2G(∇ℓ,g̃,g̃ᵀ,u,α,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent()

    end
    û, G2û_pullback
end

ng = 20
n = 4
nu = 12
nv = ng - nu
g = rand(ng,ng,n)
u = rand(nu,nu,n)
v = rand(nv,nv,n)
α = 100.0
g̃,g̃ᵀ = g2g̃(g), g2g̃ᵀ(g);
println(size(v∂gu(g̃,u,v)));
println(size(GᵀGαI(u,g̃,g̃ᵀ,α)));
println(size(y∂GᵀGx(g̃,u,u)));
x = zeros(size(u))
y = rand(nu,nu,n)
regress!(x,g̃,g̃ᵀ,α,y);
û, _, _ = G2û(g,u,α);
println(size(û));
∇ℓ = rand(nu,nu,n)
size(û2G(∇ℓ,g̃,g̃ᵀ,u,α,û))


(20, 20, 4)
(12, 12, 4)
(20, 20, 4)
(12, 12, 4)


(20, 20, 4)